In [1]:
import h3
import tqdm
import json
import numpy as np
import pandas as pd
import asyncio
import psycopg
import math
import gc

In [2]:
H3_RES = 11
HOURS = 24 * 7
TRANSIT_TYPE = 0
MAX_DISTANCE = 8

DB_CONN = "postgresql://postgres:byS*<7AxwYC#U24s@srv-captain--postgres-db-db/postgres"

In [5]:
SELECTED_H3_4 = ['841f22bffffffff']

In [26]:
# load all hexagons for a specific h3_4 area from the database

async with await psycopg.AsyncConnection.connect(DB_CONN) as conn:
    async with conn.cursor() as cursor:
        await cursor.execute("""
        select h3, h3_10, type, freq from transit
            where h3_4 = any(%s)
        """, [SELECTED_H3_4])

        hexagons = await cursor.fetchall()

In [27]:
class Row:
    h3: str
    h3_10: str
    transt_type: int
    freq: list

hexagons = np.array(hexagons, dtype=Row)

In [28]:
unique_hexagons = set(hexagons[:,0])

In [29]:
queue = asyncio.Queue()
for elem in hexagons:
    if elem[2] == TRANSIT_TYPE:
        await queue.put((h3.h3_to_parent(elem[0], 9), elem[0], elem[3], 0))

In [30]:
# define formula here
async def calc_score(origin_scores, distance, type):
    result_scores = []
    for score in origin_scores:
        if score > 1 and distance < MAX_DISTANCE:
            new_score = -0.08 * distance ** 2 + score
        elif distance > MAX_DISTANCE and score > 0.1:
            new_score = math.exp(MAX_DISTANCE - distance)
        else:
            new_score = 0
        result_scores.append(new_score)
    return result_scores

In [33]:
async def update_scores(origin_h3, current_h3, scores, distance):
    print(f"calculating {current_h3}, dist={distance + 1}", end="\r")
    # get h3 k-ring for origon_h3
    neighbors = h3.k_ring(current_h3, k= 1)
    neighbors.discard(current_h3)

    actual_neighbors = neighbors.intersection(unique_hexagons)
    return_map = {}
    for neighbor in actual_neighbors:
        score = await calc_score(scores, distance + 1, TRANSIT_TYPE)
        if max(score) >= 1:
            return_map[neighbor] = (origin_h3, score)
            await queue.put((origin_h3, neighbor, score, distance + 1))

    del(actual_neighbors)
    gc.collect()
    return return_map

In [36]:
dump = []

while not queue.empty():
    current = await queue.get()
    res = await asyncio.gather(
        update_scores(current[0], current[1], current[2], current[3])
    )

CancelledError: 